In [1]:
# this is copied from (10.12)comp_tb_tfidf-Copy1 (non-binary)

In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
xcf_comp=pd.read_csv('datasets/xcf_tb_comp_matrix.csv',index_col=0)
ars_comp=pd.read_csv('datasets/ars_tb_comp_matrix.csv',index_col=0)
kb_comp=pd.read_csv('datasets/kb_tb_comp_matrix.csv',index_col=0)

In [4]:
print(xcf_comp.shape)
print(ars_comp.shape)
print(kb_comp.shape)

(2366, 1500)
(2133, 1498)
(1864, 1507)


In [5]:
xcf_comp.head()

,recipe_id,comp_1,comp_10,comp_100,comp_1000,comp_1001,comp_1002,comp_1004,comp_1006,comp_1007,...,comp_991,comp_992,comp_993,comp_994,comp_995,comp_996,comp_997,comp_998,comp_999,class
0,9650,3,3,0,0,0,0,0,0,0,...,1,1,1,0,1,1,0,1,0,1
1,9665,5,6,0,0,0,0,0,0,0,...,1,1,1,0,1,1,0,1,0,1
2,11473,1,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
3,11478,2,2,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,11963,1,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
xcf_comp.set_index('recipe_id',inplace=True)
ars_comp.set_index('recipe_id',inplace=True)
kb_comp.set_index('recipe_id',inplace=True)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB,GaussianNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import learning_curve
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score

In [8]:
tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True)

In [9]:
def tt_tfidf_vectorizer(data,col):
    # split train and test split
    train_df,test_df,train_y,test_y=train_test_split(data.drop(col,axis=1),data[col],test_size=0.3,random_state=42)
    # transform tfidf vector
    train_X=tfidf_transformer.fit_transform(train_df)
    test_X=tfidf_transformer.transform(test_df)
    train_X=train_X.toarray()
    test_X=test_X.toarray()
    
    train_X_df=pd.DataFrame(train_X)
    test_X_df=pd.DataFrame(test_X)
    
    columns=data.columns.tolist()
    columns.remove(col)
    
    train_X_df.columns=columns
    test_X_df.columns=columns
    
    return train_X_df,test_X_df,train_y,test_y

In [10]:
xcf_train_X_df,xcf_test_X_df,xcf_train_y,xcf_test_y=tt_tfidf_vectorizer(xcf_comp,'class')
ars_train_X_df,ars_test_X_df,ars_train_y,ars_test_y=tt_tfidf_vectorizer(ars_comp,'class')
kb_train_X_df,kb_test_X_df,kb_train_y,kb_test_y=tt_tfidf_vectorizer(kb_comp,'class')

In [16]:
xcf_train_df.head()

,comp_1,comp_10,comp_100,comp_1000,comp_1001,comp_1002,comp_1004,comp_1006,comp_1007,comp_1008,...,comp_991,comp_992,comp_993,comp_994,comp_995,comp_996,comp_997,comp_998,comp_999,class
0,0.037408,0.037362,0.0,0.0,0.0,0.072293,0.0,0.0,0.0,0.0,...,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,1
1,0.041073,0.041022,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.00000,0.000000,0.0,0.069337,0.0,0.0,0.000000,0.0,1
2,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.00000,0.000000,0.0,0.075518,0.0,0.0,0.058233,0.0,1
3,0.075823,0.075730,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,1
4,0.054805,0.082106,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.069133,0.07357,0.073444,0.0,0.000000,0.0,0.0,0.000000,0.0,1


In [13]:
xcf_train_df=xcf_train_X_df.copy()
ars_train_df=ars_train_X_df.copy()
kb_train_df=kb_train_X_df.copy()

In [14]:
xcf_test_df=xcf_test_X_df.copy()
ars_test_df=ars_test_X_df.copy()
kb_test_df=kb_test_X_df.copy()

In [15]:
xcf_train_df['class']=xcf_train_y.values
ars_train_df['class']=ars_train_y.values
kb_train_df['class']=kb_train_y.values

In [33]:
xcf_test_df['class']=xcf_test_y.values
ars_test_df['class']=ars_test_y.values
kb_test_df['class']=kb_test_y.values

In [34]:
xcf_train_df.to_csv('datasets/(14.07)xcf_comp_train_df.csv')
ars_train_df.to_csv('datasets/(14.07)ars_comp_train_df.csv')
kb_train_df.to_csv('datasets/(14.07)kb_comp_train_df.csv')

In [36]:
xcf_test_df.to_csv('datasets/(14.07)xcf_comp_test_df.csv')
ars_test_df.to_csv('datasets/(14.07)ars_comp_test_df.csv')
kb_test_df.to_csv('datasets/(14.07)kb_comp_test_df.csv')

## supervised learning

In [19]:
xcf_train_X_df.head()

,comp_1,comp_10,comp_100,comp_1000,comp_1001,comp_1002,comp_1004,comp_1006,comp_1007,comp_1008,...,comp_990,comp_991,comp_992,comp_993,comp_994,comp_995,comp_996,comp_997,comp_998,comp_999
0,0.037408,0.037362,0.0,0.0,0.0,0.072293,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
1,0.041073,0.041022,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.00000,0.000000,0.0,0.069337,0.0,0.0,0.000000,0.0
2,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.00000,0.000000,0.0,0.075518,0.0,0.0,0.058233,0.0
3,0.075823,0.075730,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
4,0.054805,0.082106,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.072707,0.069133,0.07357,0.073444,0.0,0.000000,0.0,0.0,0.000000,0.0


In [20]:
# NB
def NB_pipeline(train_X,test_X,train_y,test_y):
    nb = MultinomialNB()
    cv=ShuffleSplit(n_splits=10,test_size=0.3,random_state=9527)
    nb_cv_results=cross_val_score(nb,train_X,train_y,cv=cv)
    nb.fit(train_X,train_y)
    y_pred=nb.predict(test_X)
    print('Mean Accuracy: %0.2f (+/- %0.2f)' % (nb_cv_results.mean(), nb_cv_results.std() * 2))
    print('Accuracy on test set: %0.2f' % accuracy_score(y_pred,test_y))
    return y_pred

In [21]:
# logistic regression
def log_pipeline(train_X,test_X,train_y,test_y):
    clf=LogisticRegression(random_state=9527)
    param_distribs={'C':[0.1,1,10,100,1000],
                    'solver':['lbfgs','saga']}
    log_search=RandomizedSearchCV(clf,param_distribs,n_iter=50,cv=5,n_jobs=-1)
    log_search.fit(train_X,train_y)
    print('Best parameters:',log_search.best_params_)
    print('Best estimatro:',log_search.best_estimator_)
    print('Best score:',log_search.best_score_)
    log=log_search.best_estimator_
    y_pred=log.predict(test_X)
    print('accuracy score: ',accuracy_score(y_pred,test_y))
    return y_pred

In [22]:
# random forest
def rf_pipeline(train_X,test_X,train_y,test_y):
    clf=RandomForestClassifier(random_state=42)
    param_distribs={'n_estimators':[int(x) for x in np.linspace(start = 300, stop = 500, num = 10)],
                    'max_features':['auto','sqrt','log2'],
                    'max_depth':[5,6,7,8,9,10],
                    'criterion':['gini','entropy']}
    random_search = RandomizedSearchCV(clf,param_distribs,n_iter=50,cv=5,n_jobs=-1)
    random_search.fit(train_X,train_y)
    print('Best parameters:',random_search.best_params_)
    print('Best estimatro:',random_search.best_estimator_)
    print('Best score:',random_search.best_score_)
    rf=random_search.best_estimator_
    rf.fit(train_X,train_y)
    y_pred=rf.predict(test_X)
    print('accuracy score: ',accuracy_score(y_pred,test_y))
    return y_pred

In [23]:
# xiachufang

In [24]:
xcf_ynb_pred=NB_pipeline(xcf_train_X_df,xcf_test_X_df,xcf_train_y,xcf_test_y)

Mean Accuracy: 0.58 (+/- 0.02)
Accuracy on test set: 0.60


In [25]:
xcf_ylog_pred=log_pipeline(xcf_train_X_df,xcf_test_X_df,xcf_train_y,xcf_test_y)

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

Best parameters: {'solver': 'lbfgs', 'C': 100}
Best estimatro: LogisticRegression(C=100, random_state=9527)
Best score: 0.6262057292614567
accuracy score:  0.6140845070422535


In [26]:
xcf_yrf_pred=rf_pipeline(xcf_train_X_df,xcf_test_X_df,xcf_train_y,xcf_test_y)

/opt/conda/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/conda/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/conda/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForest

Best parameters: {'n_estimators': 477, 'max_features': 'log2', 'max_depth': 8, 'criterion': 'gini'}
Best estimatro: RandomForestClassifier(max_depth=8, max_features='log2', n_estimators=477,
                       random_state=42)
Best score: 0.6412896298183671
accuracy score:  0.6577464788732394


In [27]:
ars_ynb_pred=NB_pipeline(ars_train_X_df,ars_test_X_df,ars_train_y,ars_test_y)

Mean Accuracy: 0.64 (+/- 0.04)
Accuracy on test set: 0.64


In [28]:
ars_ylog_pred=log_pipeline(ars_train_X_df,ars_test_X_df,ars_train_y,ars_test_y)

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

Best parameters: {'solver': 'saga', 'C': 1000}
Best estimatro: LogisticRegression(C=1000, random_state=9527, solver='saga')
Best score: 0.6550604924693049
accuracy score:  0.65625


In [29]:
ars_yrf_pred=rf_pipeline(ars_train_X_df,ars_test_X_df,ars_train_y,ars_test_y)

/opt/conda/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/conda/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/conda/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForest

Best parameters: {'n_estimators': 411, 'max_features': 'log2', 'max_depth': 8, 'criterion': 'gini'}
Best estimatro: RandomForestClassifier(max_depth=8, max_features='log2', n_estimators=411,
                       random_state=42)
Best score: 0.6537047428789478
accuracy score:  0.6671875


In [30]:
kb_ynb_pred=NB_pipeline(kb_train_X_df,kb_test_X_df,kb_train_y,kb_test_y)

Mean Accuracy: 0.57 (+/- 0.04)
Accuracy on test set: 0.59


In [31]:
kb_ylog_pred=log_pipeline(kb_train_X_df,kb_test_X_df,kb_train_y,kb_test_y)

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

Best parameters: {'solver': 'lbfgs', 'C': 100}
Best estimatro: LogisticRegression(C=100, random_state=9527)
Best score: 0.6227055702917771
accuracy score:  0.6535714285714286


In [32]:
kb_yrf_pred=rf_pipeline(kb_train_X_df,kb_test_X_df,kb_train_y,kb_test_y)

/opt/conda/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/conda/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/conda/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForest

Best parameters: {'n_estimators': 411, 'max_features': 'sqrt', 'max_depth': 10, 'criterion': 'entropy'}
Best estimatro: RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=411,
                       random_state=42)
Best score: 0.6218980253463012
accuracy score:  0.6375
